osconfeed.py：下载 osconfeed.json

In [4]:
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'


def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg) #1
        with urlopen(URL) as remote, open(JSON, 'wb') as local: #2
            local.write(remote.read())
    
    with open(JSON) as fp:
        return json.load(fp) #3

In [6]:
feed = load() #1

In [7]:
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [8]:
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key))

  1 conferences
484 events
357 speakers
 53 venues


In [9]:
feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

In [10]:
feed['Schedule']['speakers'][-1]['serial']

141590

In [11]:
feed['Schedule']['events'][40]['name']

'There *Will* Be Bugs'

In [12]:
feed['Schedule']['events'][40]['speakers']

[3471, 5199]

In [15]:
from collections import abc

class FrozenJson:
    """一个只读接口，使用属性表示法访问JSON类对象"""
    
    def __init__(self, mapping):
        self.__data = dict(mapping) #1
        
    def __getattr__(self, name): #2
        if hasattr(self.__data, name):
            return getattr(self.__data, name) #3
        else:
            return FrozenJson.build(self.__data[name]) #4
    
    @classmethod
    def build(cls, obj): #5
        if isinstance(obj, abc.Mapping): #6
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence): #7
            return [cls.build(item) for item in obj]
        else:
            return obj

In [20]:
grad = FrozenJson({'name': 'Jim Bo', 'class': 1982})

In [21]:
grad.class

SyntaxError: invalid syntax (<ipython-input-21-bb5c99ef29c5>, line 1)

In [18]:
getattr(grad, 'class')

1982

In [24]:
from collections import abc
import keyword

class FrozenJSON:
    """一个只读接口，使用属性表示法访问JSON类对象"""
    
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value
        
    def __getattr__(self, name): #2
        if hasattr(self.__data, name):
            return getattr(self.__data, name) #3
        else:
            return FrozenJson.build(self.__data[name]) #4
    
    @classmethod
    def build(cls, obj): #5
        if isinstance(obj, abc.Mapping): #6
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence): #7
            return [cls.build(item) for item in obj]
        else:
            return obj

In [25]:
x = FrozenJson({'2be': 'asdf'})

In [26]:
x.2be

SyntaxError: invalid syntax (<ipython-input-26-8694215ab5bd>, line 1)

In [27]:
from collections import abc

class FrozenJSON:
    """一个只读接口，使用属性表示法访问JSON类对象"""
    
    def __new__(cls, arg): #1
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls) #2
        elif isinstance(arg, abc.MutableSequence): #3
            return [cls(item) for item in arg]
        else:
            return arg
        
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value
            
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJson(self.__data[name]) #4

In [28]:
import shelve

In [29]:
db = shelve.open(DB_NAME)

NameError: name 'DB_NAME' is not defined

schedule1.py：访问保存在 shelve.Shelf 对象里的
OSCON 日程数据

In [ ]:
import warnings

DB_NAME = 'data/schedule1_db'
CONFERENCE = 'conference.115'


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs) #2
        
    
def load_db(db):
    raw_data = load() #3
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items(): #4
        record_type = collection[:-1] #5
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial']) #6
            record['serial'] = key #7
            db[key] = Record(**record) #8

schedule2.py：导入模块，定义常量和增强的 Record
类

In [ ]:
import warnings
import inspect #1

import osconfeed

DB_NAME = 'data/schedule2_db' #2
CONFERENCE = 'conference.115'


class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
    def __eq__(self, other): #3
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented

class MissingDatabaseError(RuntimeError):
    """需要数据哭但没有制定数据库时抛出""" #1
    

class DbRecord(Record): #2
    __db = None #3
    
    @staticmethod #4
    def set_db(db):
        DbRecord.__db = db #5
    
    @staticmethod #6
    def get_db():
        return DbRecordc.__db
    
    @classmethod #7
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident] #8
        except TypeError:
            if db is None: #9
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else: #10
                raise
    
    def __repr__(self):
        if hasattr(self, 'serial'): #11
            cls_name = self.__class__.__name__
            return '<{} serial={!r}>'.format(cls_name, self.serial)
        else:
            return super().__repr__() #12
        

class Event(DbRecord): #1
    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial)
        return self.__class__.fetch(key) #2
    
    @property
    def speakers(self):
        if not hasattr(self, '_speaker_objs'): #3
            spkr_serials = self.__dict__['speakers'] #4
            fetch = self.__class__.fetch #5
            self._speaker_objs = [fetch('speaker.{}'.format(key)) for key in spkr_serials] #6
            
        return self._speaker_objs #7
    
    def __repr__(self):
        if hasattr(self, 'name'): #8
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__() #9
        
def load_db(db):
    raw_data = osconfeed.load()
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1] #1
        cls_name = record_type.capitalize() #2
        cls = globals().get(cls_name, DbRecord) #3
        if inspect.isclass(cls) and issubclass(cls, DbRecord): #4
            factory = cls #5
        else:
            factory = DbRecord #6
        for record in rec_list: #7
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = factory(**record) #8

In [ ]:
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

In [1]:
class LineItem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self):
        return self.__weight
    
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

In [3]:
x = LineItem('x', 1, 10.00)

In [4]:
x

In [5]:
x.weight

1

In [6]:
class Class:
    data = 'the class data attr'
    @property
    def prop(self):
        return 'the prop value'

In [7]:
obj = Class()

In [8]:
obj

In [9]:
vars(obj)

{}

In [10]:
obj.data

'the class data attr'

In [11]:
obj.data = 'bar'

In [12]:
vars(obj)

{'data': 'bar'}

In [13]:
obj.data

'bar'

In [14]:
Class.data

'the class data attr'

In [15]:
Class.prop

In [16]:
obj.prop

'the prop value'

In [17]:
obj.prop = 'foo'

AttributeError: can't set attribute

In [18]:
obj.__dict__['prop'] = 'foo'

In [19]:
vars(obj)

{'data': 'bar', 'prop': 'foo'}

In [20]:
obj.prop

'the prop value'

In [22]:
Class.prop = 'baz'

In [23]:
Class.prop

'baz'

In [24]:
obj.prop

'foo'

In [25]:
vars(Class)

mappingproxy({'__module__': '__main__',
              'data': 'the class data attr',
              'prop': 'baz',
              '__dict__': <attribute '__dict__' of 'Class' objects>,
              '__weakref__': <attribute '__weakref__' of 'Class' objects>,
              '__doc__': None})

In [26]:
aa = Class()

In [27]:
vars(aa)

{}

In [28]:
aa.prop

'baz'

In [29]:
obj.data

'bar'

In [30]:
Class.data

'the class data attr'

In [31]:
Class.data = property(lambda self: 'the "data" prop value')

In [32]:
obj.data

'the "data" prop value'

In [33]:
Class.data

In [34]:
del Class.data

In [35]:
obj.data

'bar'

In [36]:
class Foo:
    @property
    def bar(self):
        '''The bar attribute'''
        return self.__dict__['bar']
    
    @bar.setter
    def bar(self, value):
        self.__dict__['bar'] = value

In [37]:
help(Foo)

Help on class Foo in module __main__:

class Foo(builtins.object)
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  bar
 |      The bar attribute



In [38]:
help(Foo.bar)

Help on property:

    The bar attribute



bulkfood_v2prop.py：使用特性工厂函数 quantity

In [41]:
class LineItem:
    weight = quantity('weight')
    price = quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
        
    def subtotal(self):
        return self.weight * self.price

bulkfood_v2prop.py：quantity 特性工厂函数

In [40]:
def quantity(storage_name): #1
    def qty_getter(instance): #2
        return instance.__dict__[storage_name] #3
    
    def qty_setter(instance, value): #4
        if value > 0:
            instance.__dict__[storage_name] = value #5
        else:
            raise ValueError('value must be > 0')
            
    return property(qty_getter, qty_setter) #6

In [42]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)

In [43]:
nutmeg.weight, nutmeg.price

(8, 13.95)

In [44]:
sorted(vars(nutmeg).items())

[('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]

blackknight.py：灵感来自电影《巨蟒与圣杯》中的黑
衣骑士角色

In [45]:
class BlackKnight:
    def __init__(self):
        self.members = ['an arm', 'another arm', 'a leg', 'another leg']
        self.phrases = ["'Tis but a scratch.'", "It's just a flesh wound.", "I'm invinvible!", "All right, we'll call it a draw."]
        
    @property
    def member(self):
        print('next member is:')
        return self.members[0]
    
    @member.deleter
    def member(self):
        text = 'BLACK KNIGHT (loses {})\n-- {}'
        print(text.format(self.members.pop(0), self.phrases.pop(0)))

In [46]:
knight = BlackKnight()

In [47]:
knight.member

next member is:


'an arm'

In [48]:
del knight.member

BLACK KNIGHT (loses an arm)
-- 'Tis but a scratch.'


In [49]:
del knight.member

BLACK KNIGHT (loses another arm)
-- It's just a flesh wound.


In [50]:
del knight.member

BLACK KNIGHT (loses a leg)
-- I'm invinvible!


In [51]:
del knight.member

BLACK KNIGHT (loses another leg)
-- All right, we'll call it a draw.
